In [1]:
!pip install replicate
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from google.colab import userdata
import os

#ambil api
api_token = userdata.get("api_token")
os.environ["REPLICATE_API_TOKEN"] = api_token

In [3]:
from langchain_community.llms import Replicate

#kuchiyose model
model = Replicate(model="ibm-granite/granite-3.3-8b-instruct")

In [4]:
# cek
response = model.invoke("halo granite, tolong jelaskan apa itu berita palsu (fake news) dalam 2 kalimat.")
print(response)


Berita palsu, juga dikenal sebagai fake news, adalah informasi yang disebarkan sebagai berita tetapi tidak berasal dari sumber yang terpercaya atau tidak dapat diberi kesan. Ini biasanya berisi ketidakbenaran, dramatisasi, atau konteks yang telah diubah dengan tujuan untuk menyebarkan ideologi tertentu atau menciptakan kerapuan.


In [7]:
import pandas as biji

#memasukkan dataset fek dan tru, pake py, lalu on_bad_lines supaya baris rusak dilewati
fake_df = biji.read_csv("/content/Fake.csv", engine="python", on_bad_lines='skip')
true_df = biji.read_csv("/content/True.csv", engine="python", on_bad_lines='skip')

#mengambil 3 cnth berita fek dan 3 berita rill
samples = []
for i in range(3):
    samples.append(("FAKE", fake_df.iloc[i]["text"]))
    samples.append(("REAL", true_df.iloc[i]["text"]))

results = []    #list untuk menyimpan hasil dai granite

In [8]:
from langchain_community.llms import Replicate

#parameter utk mengontrol granite
parameters = {
    "top_k": 50,                                  #50 kata teratas
    "top_p": 0.9,                                 #ngambil 90% probabiitas teratas
    "max_tokens": 256,                            #maks jawaban
    "min_tokens": 0,                              #min
    "random_seed": 42,                            #biar bisa direplikasi
    "repetition_penalty": 1.1,                    #penalti kalau ngulang
    "stopping_criteria": "length (256 tokens)",
    "stopping_sequence": None
}

#menyiapkan granite dari replicatee dgn parameter diatas
model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    model_kwargs=parameters
)


In [9]:
for idx, (true_label, text) in enumerate(samples, start=1):
    #promptnya
    prompt = f"""
    classify the following news article.
    only classify as real if you are absolutely sure the content can be verified by credible news outlets. otherwise, classify as fake.

    return only JSON with two keys: "label" and "reason".

    news:
    {text}
    """

    #panggil granite
    output = model.invoke(prompt)

    #print hasil di console
    print(f"\n--- Sample {idx} ---")
    print("Granite output:", output)

    #simpan ke ;list hasil
    results.append({
        "true_label": true_label,
        "granite_output": output
    })

#simpan hasil ke dataframe
df_results = biji.DataFrame(results)
print("\n=== Ringkasan Hasil Granite ===")
print(df_results)



--- Sample 1 ---
Granite output: {
  "label": "real",
  "reason": "The news article presents a verified tweet from the official account of former U.S. President Donald Trump (December 31, 2017) and subsequent reactions from Twitter users. The content can be cross-referenced with Twitter's archive and news reports from credible outlets at the time. The article describes an event that occurred in the real world and is thus classified as real."
}

--- Sample 2 ---
Granite output: {
  "label": "real",
  "reason": "The news article is a report from Reuters, a well-established and credible international news agency. It discusses ongoing political events and statements made by U.S. politicians regarding budget negotiations, tax cuts, military spending, and immigration policy, all of which can be verified through other reputable news sources."
}

--- Sample 3 ---
Granite output: {
  "label": "real",
  "reason": "The news article is a summary of events that have been reported by credible news 

In [10]:
import json

#parsing hasil granite (dari json string ke kolom dataframe)
df_results["predicted_label"] = df_results["granite_output"].apply(
    lambda x: json.loads(x)["label"] if isinstance(x, str) else None
)

df_results["reason"] = df_results["granite_output"].apply(
    lambda x: json.loads(x)["reason"] if isinstance(x, str) else None
)

#lihat hasil parsing
print("\n=== Hasil Setelah Parsing ===")
print(df_results[["true_label", "predicted_label", "reason"]])

#hitung akurasi sederhana granite dgn data kaggle
accuracy = (df_results["true_label"] == df_results["predicted_label"]).mean()
print(f"\nAccuracy Granite vs Dataset Kaggle: {accuracy:.2f}")



=== Hasil Setelah Parsing ===
  true_label predicted_label  \
0       FAKE            real   
1       REAL            real   
2       FAKE            real   
3       REAL            real   
4       FAKE            real   
5       REAL            real   

                                              reason  
0  The news article presents a verified tweet fro...  
1  The news article is a report from Reuters, a w...  
2  The news article is a summary of events that h...  
3  This news article is real as it is based on a ...  
4  The news article describes a verifiable event,...  
5  The news article is a report from Reuters, a c...  

Accuracy Granite vs Dataset Kaggle: 0.00
